In [82]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import json

In [83]:
EPS = 1e-8

def normalize(X):
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    return (X - X_mean) / (X_std + EPS), X_mean, X_std

def denormalize(X_normalized, X_mean, X_std):
    return (X_normalized * (X_std + EPS)) + X_mean

In [84]:
with open("../data/labeled_data_2.json", "r") as json_file:
    data = json.load(json_file)

y = np.array([entry['score'] for entry in data.values()])
y_avg = np.mean(y)
y = np.where(y_avg < y, y, 0)
y = np.where(y_avg >= y, y, 1)
X = np.array([np.array(list({k : v for k, v in entry.items() if k != 'score'}.values())) for entry in data.values()])
X_normalized, X_mean, X_std = normalize(X)
X_train, X_eval, y_train, y_eval = train_test_split(X_normalized, y, test_size=0.05)

# 1. Random Forest / Bagging

In [85]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100)

# Train the model on the training data
rf_classifier.fit(X_train, y_train)

# Make predictions on the evaluation data
rf_predictions = rf_classifier.predict(X_eval)

# Evaluate the model's performance
rf_accuracy = accuracy_score(y_eval, rf_predictions)
print(f"Random Forest Classifier Accuracy: {rf_accuracy:.2f}")
print(classification_report(y_eval, rf_predictions))

Random Forest Classifier Accuracy: 0.86
              precision    recall  f1-score   support

         0.0       1.00      0.75      0.86         4
         1.0       0.75      1.00      0.86         3

    accuracy                           0.86         7
   macro avg       0.88      0.88      0.86         7
weighted avg       0.89      0.86      0.86         7



In [86]:
# Initialize the base estimator (e.g., Decision Tree)
base_estimator = DecisionTreeClassifier()

# Initialize the Bagging Classifier
bagging_classifier = BaggingClassifier(base_estimator=base_estimator, n_estimators=100)

# Train the model on the training data
bagging_classifier.fit(X_train, y_train)

# Make predictions on the evaluation data
bagging_predictions = bagging_classifier.predict(X_eval)

# Evaluate the model's performance
bagging_accuracy = accuracy_score(y_eval, bagging_predictions)
print(f"Bagging Classifier Accuracy: {bagging_accuracy:.2f}")
print(classification_report(y_eval, bagging_predictions))

/Users/ryan/miniforge3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Bagging Classifier Accuracy: 0.71
              precision    recall  f1-score   support

         0.0       1.00      0.50      0.67         4
         1.0       0.60      1.00      0.75         3

    accuracy                           0.71         7
   macro avg       0.80      0.75      0.71         7
weighted avg       0.83      0.71      0.70         7



# 2. Pruning data with PCA first

In [90]:
# Initialize the PCA object with 5 principal components
pca = PCA(n_components=1)

# Fit and transform the training data
X_train_pca = pca.fit_transform(X_train)

# Transform the evaluation data using the same PCA model
X_eval_pca = pca.transform(X_eval)

In [91]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100)

# Train the model on the training data
rf_classifier.fit(X_train_pca, y_train)

# Make predictions on the evaluation data
rf_predictions = rf_classifier.predict(X_eval_pca)

# Evaluate the model's performance
rf_accuracy = accuracy_score(y_eval, rf_predictions)
print(f"Random Forest Classifier Accuracy: {rf_accuracy:.2f}")
print(classification_report(y_eval, rf_predictions))

Random Forest Classifier Accuracy: 0.57
              precision    recall  f1-score   support

         0.0       0.67      0.50      0.57         4
         1.0       0.50      0.67      0.57         3

    accuracy                           0.57         7
   macro avg       0.58      0.58      0.57         7
weighted avg       0.60      0.57      0.57         7



In [92]:
# Initialize the base estimator (e.g., Decision Tree)
base_estimator = DecisionTreeClassifier()

# Initialize the Bagging Classifier
bagging_classifier = BaggingClassifier(base_estimator=base_estimator, n_estimators=100)

# Train the model on the training data
bagging_classifier.fit(X_train_pca, y_train)

# Make predictions on the evaluation data
bagging_predictions = bagging_classifier.predict(X_eval_pca)

# Evaluate the model's performance
bagging_accuracy = accuracy_score(y_eval, bagging_predictions)
print(f"Bagging Classifier Accuracy: {bagging_accuracy:.2f}")
print(classification_report(y_eval, bagging_predictions))

Bagging Classifier Accuracy: 0.57
              precision    recall  f1-score   support

         0.0       0.67      0.50      0.57         4
         1.0       0.50      0.67      0.57         3

    accuracy                           0.57         7
   macro avg       0.58      0.58      0.57         7
weighted avg       0.60      0.57      0.57         7



/Users/ryan/miniforge3/lib/python3.9/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
